In [ ]:
from transformers import pipeline

ner = pipeline("ner", model = 'dbmdz/bert-large-cased-finetuned-conll03-english', grouped_entities=True, device='cuda')
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/home/slakter/miniconda3/envs/venv/lib/python3.11/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", device='cuda')


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

In [ ]:
from torch import tensor
import torch.nn.functional as F
import torch
import numpy as np
from fastcore.test import *

In [ ]:
tokenizer = summarizer.tokenizer
model_emb = summarizer.model.model.encoder.embed_tokens

In [ ]:
@torch.no_grad()
def emb(word):
    return model_emb(tensor(tokenizer(word)['input_ids'][1:-1]).cuda()).sum(0)[None]


def unemb(embedding, p=2.0):
    token = F.pairwise_distance(
        embedding, model_emb.get_parameter('weight'), p=p).argmin()
    return tokenizer.decode(token)

In [ ]:
unemb(emb("queen") + emb("man") - emb("woman"))

'que'

In [ ]:
unemb(emb("king") - emb("man") + emb("woman"))

'king'

In [ ]:
def unemb_cos(embedding):
    token = F.cosine_similarity(embedding, model_emb.get_parameter('weight')).argmax()
    return tokenizer.decode(token)

In [ ]:
unemb_cos(emb("king") - emb("man") + emb("woman"))

'king'

In [ ]:
unemb_cos(emb("queen") + emb("man") - emb("woman"))

'en'

## try decoder model

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to build something beautiful and cool in a way that is not just one game, but a way of life'},
 {'generated_text': 'In this course, we will teach you how to best use your skills to perform the same task.\n\n\n\n\nWhat are some tools you'}]

In [ ]:
head = generator.model.lm_head

In [ ]:
@torch.no_grad()
def emb(word):
    return generator.model.transformer(tensor(generator.tokenizer(word)['input_ids']))['last_hidden_state'].sum(0)[None]

def unemb(embedding, p=2.0):
    token = head(embedding).argmax()
    return tokenizer.decode(token)